# CHESS RL

In [1]:
import os
import sys

In [2]:
import random

In [3]:
import time

In [4]:
import pygame
import chess

pygame 2.3.0 (SDL 2.24.2, Python 3.10.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
import gym
import chess
import chess.pgn
import chess.engine
import numpy as np

In [6]:
def get_piece_value(piece):
    # Asignar un valor numérico a cada pieza
    if piece.piece_type == chess.PAWN:
        return 1
    elif piece.piece_type == chess.KNIGHT:
        return 3
    elif piece.piece_type == chess.BISHOP:
        return 3
    elif piece.piece_type == chess.ROOK:
        return 5
    elif piece.piece_type == chess.QUEEN:
        return 9
    elif piece.piece_type == chess.KING:
        return 0
    
    return 0

In [7]:
# class ChessEnv(gym.Env):
#     metadata = {'render.modes': ['human']}
    
#     def __init__(self):
#         self.board = chess.Board()
#         self.action_space = gym.spaces.Discrete(4096)
#         self.observation_space = gym.spaces.Box(low=0, high=1, shape=(64,))
        
#     def reset(self):
#         self.board.reset()
#         return self.get_observation()
    

#     # # Esta función hacef que si encuentra un movimiento ilegal pierda la partida
#     # def step(self, action):
#     #     move = self.get_move_from_action(action)
#     #     if move not in self.board.legal_moves:
#     #         print("Movimiento ilegal: {}".format(move))
#     #         return self.get_observation(), -1, True, {}
#     #     self.board.push(move)
#     #     observation = self.get_observation()
#     #     reward = self.get_reward()
#     #     done = self.board.is_game_over()
#     #     if done:
#     #         if reward == 0:
#     #             # empate
#     #             reward = -0.5
#     #         elif reward == 1:
#     #             # ganó el agente
#     #             reward = 1
#     #         else:
#     #             # perdió el agente
#     #             reward = -1
#     #     info = {}
#     #     return observation, reward, done, info

#     # Esta función hace que no pierda inmediatamente
#     def get_legal_action(model, input, board):
#         legal_moves = list(board.legal_moves)
#         action = model.predict(input)[0]
        
#         while action not in legal_moves:
#             action = np.random.randint(model.action_space.n)
        
#         return action

#     def step(self, action):
#         move = self.get_move_from_action(action)
        
#         if move not in self.board.legal_moves:
#             move = self.get_legal_move()
        
#         self.board.push(move)
#         observation = self.get_observation()
#         reward = self.get_reward()
#         done = self.board.is_game_over()
#         info = {}
#         return observation, reward, done, info


    
#     def render(self, mode='human'):
#         print(self.board)
        
#     def get_observation(self):
#         # Convertir el tablero en una matriz de entrada
#         input = np.zeros((64,))
#         for i in range(64):
#             piece = self.board.piece_at(i)
#             if piece is not None:
#                 input[i] = get_piece_value(piece)

#         return input
        
#     # def get_move_from_action(self, action):
#     #     move = chess.Move()
#     #     move.from_square = action % 64
#     #     move.to_square = action // 64
#     #     return move
#     def get_move_from_action(self, action):
#         from_square = action % 64
#         to_square = action // 64
#         return chess.Move(from_square=from_square, to_square=to_square)

    
#     def get_action_from_move(self, move):
#         return move.from_square + move.to_square * 64
    
#     def get_reward(self):
#         if self.board.is_checkmate():
#             if self.board.turn == chess.WHITE:
#                 return -1
#             else:
#                 return 1
#         elif self.board.is_stalemate() or self.board.is_insufficient_material() or self.board.is_seventyfive_moves() or self.board.is_fivefold_repetition():
#             return 0.5
#         else:
#             return 0
    
#     # def get_output(model, input, board):
#     #     # Obtener la salida del modelo
#     #     output, _ = model.predict(input)
        
#     #     # Obtener la lista de jugadas legales
#     #     legal_moves = list(board.legal_moves)
        
#     #     # Verificar si la acción producida por el modelo es una jugada legal
#     #     if output >= len(legal_moves):
#     #         # Si no es legal, tomar una acción aleatoria
#     #         return legal_moves[random.randint(0, len(legal_moves) - 1)]
        
#     #     # Obtener la mejor jugada posible
#     #     best_move = legal_moves[output]
        
#     #     # Verificar si la jugada es legal en el tablero actual
#     #     if best_move not in legal_moves:
#     #         # Si no es legal, tomar una acción aleatoria
#     #         return legal_moves[random.randint(0, len(legal_moves) - 1)]
        
#     #     return best_move

#     def get_output(model, input, board):
#         legal_moves = board.get_legal_action()
#         if legal_moves is None:
#             return None
        
        
#         # Obtener la salida del modelo
#         output, _ = model.predict(input)
        
#         # Obtener la lista de jugadas legales
#         legal_moves = list(board.legal_moves)
        
#         # Verificar si la acción producida por el modelo es una jugada legal
#         if legal_moves:
#             if output >= len(legal_moves):
#                 # Si no es legal, tomar una acción aleatoria
#                 return random.choice(legal_moves)
#             else:
#                 # Devolver la mejor jugada posible
#                 return legal_moves[output]
#         else:
#             # Si no hay movimientos legales, devolver None
#             return None

In [8]:
class ChessEnv(gym.Env):
    metadata = {'render.modes': ['human']}
    
    def __init__(self):
        self.board = chess.Board()
        self.action_space = gym.spaces.Discrete(4096)
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(64,))
        self.episode_rewards = []
        
    def reset(self):
        self.board.reset()
        return self.get_observation()
    
    def step(self, action):
        move = self.get_move_from_action(action)
        while move not in self.board.legal_moves:
            action = self.action_space.sample()
            move = self.get_move_from_action(action)
        self.board.push(move)
        observation = self.get_observation()
        reward = self.get_reward()
        self.episode_rewards.append(reward)
        done = self.board.is_game_over()
        info = {}
        return observation, reward, done, info
    
    def render(self, mode='human'):
        print(self.board)
        
    def get_observation(self):
        # Convertir el tablero en una matriz de entrada
        # input = np.zeros((64,))
        input = np.zeros((1, 64))
        for i in range(64):
            piece = self.board.piece_at(i)
            if piece is not None:
                # input[i] = get_piece_value(piece)
                input[0, i] = get_piece_value(piece)


        return input
        
    def get_move_from_action(self, action):
        from_square = action % 64
        to_square = action // 64
        move = chess.Move(from_square=from_square, to_square=to_square)
        return move

    
    def get_legal_action(self):
        legal_moves = list(self.board.legal_moves)
        if len(legal_moves) == 0:
            return None
        action = self.get_action_from_move(legal_moves[0])
        return action
    
    def get_action_from_move(self, move):
        return move.from_square + move.to_square * 64
    
    def get_reward(self):
        if self.board.is_checkmate():
            if self.board.turn == chess.WHITE:
                return -1
            else:
                return 1
        elif self.board.is_stalemate() or self.board.is_insufficient_material() or self.board.is_seventyfive_moves() or self.board.is_fivefold_repetition():
            return 0.5
        else:
            return 0


In [9]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

In [10]:
# Crear el entorno
env = ChessEnv()

In [11]:
# Crear el entorno vectorizado
env = make_vec_env(lambda: env, n_envs=1)

In [12]:
# Crear el modelo
model = PPO("MlpPolicy", env, verbose=1)

Using cuda device


In [13]:
from stable_baselines3.common.callbacks import BaseCallback

class RewardCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(RewardCallback, self).__init__(verbose)
        self.epoch_rewards = []

    def _on_epoch_end(self) -> bool:
        mean_reward = np.mean(self.epoch_rewards)
        print("Mean reward:", mean_reward)
        self.epoch_rewards = []
        return True

    def _on_rollout_end(self) -> None:
        episode_rewards = self.training_env.get_attr('episode_rewards')
        self.epoch_rewards.extend(episode_rewards)
        
    def _on_step(self) -> bool:
        return True


In [14]:
callback = RewardCallback()

In [15]:
# # Entrenar el modelo
# model.learn(total_timesteps=10000)
model.learn(total_timesteps=10000, callback=callback)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 462      |
|    ep_rew_mean     | 0.5      |
| time/              |          |
|    fps             | 225      |
|    iterations      | 1        |
|    time_elapsed    | 9        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 501         |
|    ep_rew_mean          | 0.5         |
| time/                   |             |
|    fps                  | 207         |
|    iterations           | 2           |
|    time_elapsed         | 19          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.030578867 |
|    clip_fraction        | 0.514       |
|    clip_range           | 0.2         |
|    entropy_loss         | -8.32       |
|    explained_variance   | -1.6        |
|    learning_rate        | 0.

In [16]:
# Guardar el modelo entrenado
model.save("chess_model")

In [17]:
# Cargar el modelo entrenado
loaded_model = PPO.load("chess_model")

In [18]:
PATH_ENGINE = 'L:/ENGINES/komodo-13_201fd6/Windows/komodo-13.02-64bit.exe'

In [19]:
# Crear una instancia del motor de ajedrez Stockfish
engine = chess.engine.SimpleEngine.popen_uci(PATH_ENGINE)

In [20]:
# env = ChessEnv()
# obs = env.reset()
# print(obs.shape)

In [21]:
# def board_to_observation(board):
#     # Convertir el tablero en una matriz de entrada
#     input = np.zeros((64,))
#     for i in range(64):
#         piece = board.piece_at(i)
#         if piece is not None:
#             input[i] = get_piece_value(piece)

#     return input


In [22]:
# # Crear un tablero virtual
# board = chess.Board()

# # Hacer que el modelo juegue contra el motor de ajedrez
# while not board.is_game_over():
#     if board.turn == chess.WHITE:
#         # Turno del modelo
#         action, _ = model.predict([board_to_observation(board)])
#         move = chess.Move.from_uci(str(action))
#         board.push(move)
#     else:
#         # Turno del motor de ajedrez
#         result = engine.play(board, chess.engine.Limit(time=2.0))
#         board.push(result.move)

# # Cerrar el motor de ajedrez
# engine.quit()

In [23]:
def board_to_observation(board):
    board_vector = np.zeros(64, dtype=np.int8)

    for i in range(64):
        piece = board.piece_at(i)
        if piece is not None:
            board_vector[i] = {'P': 1, 'N': 2, 'B': 3, 'R': 4, 'Q': 5, 'K': 6,
                               'p': -1, 'n': -2, 'b': -3, 'r': -4, 'q': -5, 'k': -6}[piece.symbol()]

    if board.turn == chess.BLACK:
        board_vector = -board_vector

    return board_vector.flatten()


In [24]:
# def action_to_move(action, board):
#     action = int(action)
#     legal_moves = list(board.legal_moves)
#     move = legal_moves[action]
#     return move

def action_to_move(action, board):
    legal_moves = list(board.legal_moves)
    if action >= len(legal_moves):
        action = len(legal_moves) - 1
    move = legal_moves[action]
    return move


In [ ]:
# Inicializar Pygame
pygame.init()

# Definir el tamaño de la ventana
window_size = (600, 600)
window = pygame.display.set_mode(window_size)

# Crear una instancia del motor de ajedrez Stockfish
engine = chess.engine.SimpleEngine.popen_uci(PATH_ENGINE)

# Cargar las imágenes de las piezas
piece_images = {}
for color in [chess.WHITE, chess.BLACK]:
    for piece_type in [chess.PAWN, chess.KNIGHT, chess.BISHOP, chess.ROOK, chess.QUEEN, chess.KING]:
        piece = chess.Piece(piece_type, color)
        if color == chess.WHITE:
            filename = f"IMAGES/{piece.symbol()}.png"
        else:
            filename = f"IMAGES/{piece.symbol().lower()}_b.png"
        image = pygame.image.load(filename)
        image = pygame.transform.scale(image, (50, 50))
        piece_images[piece] = image

# Cargar la imagen del tablero
board_image = pygame.image.load("IMAGES/board.png")
board_image = pygame.transform.scale(board_image, (500, 500))

# Crear un tablero virtual
board = chess.Board()

# Crear una lista para almacenar las jugadas realizadas
moves_list = []

# Hacer que el modelo juegue contra el motor de ajedrez
while not board.is_game_over():
    if board.turn == chess.WHITE:
        # Turno del modelo
        action, _ = model.predict([board_to_observation(board)])
        move = action_to_move(int(action), board)
        board.push(move)
    else:
        # Turno del motor de ajedrez
        result = engine.play(board, chess.engine.Limit(time=2.0))
        board.push(result.move)

    # Añadir la jugada al final de la lista de jugadas realizadas
    moves_list.append(str(move))

    # Dibujar el tablero y las piezas
    window.blit(board_image, (50, 50))
    for square, piece in board.piece_map().items():
        x, y = chess.square_file(square) * 62.5 + 50, (7 - chess.square_rank(square)) * 62.5 + 50
        window.blit(piece_images[piece], (x, y))

    # Dibujar la lista de jugadas realizadas
    moves_text = ""
    for i, move in enumerate(moves_list):
        if i % 2 == 0:
            moves_text += f"{i // 2 + 1}. {move} "
        else:
            moves_text += f"{move} "
    font = pygame.font.SysFont("Arial", 18)
    text_surface = font.render(moves_text, True, pygame.Color("black"))
    window.blit(text_surface, (50, 10))

    # Actualizar la ventana
    pygame.display.flip()

    # Comprobar si se ha pulsado el botón de retroceder jugada
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_z and pygame.key.get_mods() & pygame.KMOD_CTRL:
                # Retroceder una jugada
                if len(history) > 0:
                    board.pop()
                    history.pop()
            elif event.key == pygame.K_r and pygame.key.get_mods() & pygame.KMOD_CTRL:
                # Reiniciar la partida
                board.reset()
                history.clear()
            elif event.key == pygame.K_q:
                # Salir del juego
                pygame.quit()
                sys.exit()
                
    # Dibujar el tablero y las piezas
    window.blit(board_image, (0, 0))
    for square, piece in board.piece_map().items():
        x, y = chess.square_file(square) * square_size, (7 - chess.square_rank(square)) * square_size
        window.blit(piece_images[piece], (x, y))
        
    # Actualizar la ventana
    pygame.display.flip()

In [ ]:
# Mostrar los resultados de la partida
result = board.result()
print(f"Resultado de la partida: {result}")


In [ ]:
# Mostrar lista de jugadas
print("Lista de jugadas:")
for i, move in enumerate(moves_list):
    if i % 2 == 0:
        print(f"{i//2+1}. {move}")
    else:
        print(f"    {move}")


In [25]:
# Inicializar Pygame
pygame.init()

# Definir el tamaño de la ventana
window_size = (512, 512)
window = pygame.display.set_mode(window_size)

# Cargar las imágenes de las piezas
piece_images = {}
for color in [chess.WHITE, chess.BLACK]:
    for piece_type in [chess.PAWN, chess.KNIGHT, chess.BISHOP, chess.ROOK, chess.QUEEN, chess.KING]:
        piece = chess.Piece(piece_type, color)
        if color == chess.WHITE:
            filename = f"IMAGES/{piece.symbol()}.png"
        else:
            filename = f"IMAGES/{piece.symbol().lower()}_b.png"
        image = pygame.image.load(filename)
        image = pygame.transform.scale(image, (32, 32))
        piece_images[piece] = image
# Cargar la imagen del tablero
b_image = pygame.image.load("IMAGES/board.png")
board_image = pygame.transform.scale(b_image, (500, 500))


# Crear un tablero virtual
board = chess.Board()

# Hacer que el modelo juegue contra el motor de ajedrez
while not board.is_game_over():
    if board.turn == chess.WHITE:
        # Turno del modelo
        action, _ = model.predict([board_to_observation(board)])
        move = action_to_move(int(action), board)
        board.push(move)
    else:
        # Turno del motor de ajedrez
        result = engine.play(board, chess.engine.Limit(time=2.0))
        board.push(result.move)
    
    # Dibujar el tablero y las piezas
    window.blit(board_image, (0, 0))
    for square, piece in board.piece_map().items():
        x, y = chess.square_file(square) * 64, (7 - chess.square_rank(square)) * 64
        window.blit(piece_images[piece], (x, y))
    
    # Actualizar la ventana
    pygame.display.flip()

# Cerrar el motor de ajedrez
engine.quit()

# Salir de Pygame
pygame.quit()

In [26]:
# play_game(loaded_model)

In [27]:
# Inicializar Pygame
pygame.init()

# Definir el tamaño de la ventana
window_size = (512, 512)
window = pygame.display.set_mode(window_size)

# Cargar las imágenes de las piezas
piece_images = {}
for color in [chess.WHITE, chess.BLACK]:
    for piece_type in [chess.PAWN, chess.KNIGHT, chess.BISHOP, chess.ROOK, chess.QUEEN, chess.KING]:
        piece = chess.Piece(piece_type, color)
        if color == chess.WHITE:
            filename = f"IMAGES/{piece.symbol()}.png"
        else:
            filename = f"IMAGES/{piece.symbol().lower()}_b.png"
        image = pygame.image.load(filename)
        image = pygame.transform.scale(image, (32, 32))
        piece_images[piece] = image

# Cargar la imagen del tablero
b_image = pygame.image.load("IMAGES/board.png")
board_image = pygame.transform.scale(b_image, (500, 500))

# Definir la posición de las piezas
piece_positions = {square: (chess.square_file(square) * 62 + 25, (7 - chess.square_rank(square)) * 62 + 25) for square in range(64)}

# Definir la pieza seleccionada por el usuario (inicialmente ninguna)
selected_piece = None

# Crear un tablero virtual
board = chess.Board()

# Hacer que el modelo juegue contra el motor de ajedrez
while not board.is_game_over():
    if board.turn == chess.WHITE:
        # Turno del modelo
        action, _ = model.predict([board_to_observation(board)])
        move = action_to_move(int(action), board)
        board.push(move)
    else:
        # Turno del usuario
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1:  # Botón izquierdo del ratón
                    x, y = event.pos
                    file = (x - 25) // 62
                    rank = 7 - ((y - 25) // 62)
                    square = chess.square(file, rank)
                    piece = board.piece_at(square)
                    if piece is not None and piece.color == chess.WHITE:
                        # Resaltar la pieza seleccionada y sus posibles movimientos
                        selected_piece = piece
                        legal_moves = board.legal_moves
                        legal_moves_for_piece = [move for move in legal_moves if move.from_square == square]
                        highlighted_squares = set([move.to_square for move in legal_moves_for_piece])
                    else:
                        # Deseleccionar la pieza
                        selected_piece = None
                        highlighted_squares = set()

        # Obtener la posición del mouse
        mouse_x, mouse_y = pygame.mouse.get_pos()

        file_index = int(mouse_x / 62)
        rank_index = 7 - int(mouse_y / 62)

        if 0 <= file_index < 8 and 0 <= rank_index < 8:
            mouse_square = chess.square(file_index + ord('a'), rank_index)
            
            # Resaltar la casilla sobre la que está el mouse
            if chess.square_file(mouse_square) < 8 and chess.square_rank(mouse_square) < 8:
                pygame.draw.rect(window, (255, 0, 0), (chess.square_file(mouse_square) * 62,
                                                        (7 - chess.square_rank(mouse_square)) * 62, 62, 62), 4)
        else:
            mouse_square = None

            
        # Obtener la posición del mouse
        mouse_x, mouse_y = pygame.mouse.get_pos()
        mouse_square = chess.square(chess.FILE_NAMES.index(chr(int(mouse_x / 62) + ord('a'))),
                                    7 - int(mouse_y / 62))

        # Resaltar la casilla sobre la que está el mouse
        if chess.square_file(mouse_square) < 8 and chess.square_rank(mouse_square) < 8:
            pygame.draw.rect(window, (255, 0, 0), (chess.square_file(mouse_square) * 62,
                                                    (7 - chess.square_rank(mouse_square)) * 62, 62, 62), 4)

        # Actualizar la ventana
        pygame.display.flip()

        # Detectar eventos del mouse
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.MOUSEBUTTONDOWN:
                # Obtener la casilla en la que se hizo clic
                click_x, click_y = event.pos
                click_square = chess.square(chess.FILE_NAMES.index(chr(int(click_x / 62) + ord('a'))),
                                            7 - int(click_y / 62))
                
                # Mover la pieza si la casilla clicada es una casilla válida
                if click_square in legal_moves:
                    move = chess.Move(mouse_square, click_square)
                    board.push(move)
                    legal_moves = [move for move in board.legal_moves]
                    engine.position(board)
                    break


ValueError: 'i' is not in list

: 